In [1]:
using Revise
import SuperconductingCavities as SC

using YAXArrays
using DimensionalData

using Logging
using MiniLoggers


ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.


In [2]:
# MiniLogger(minlevel = MiniLoggers.Info) |> global_logger
InfoLogger = MiniLogger(minlevel = MiniLoggers.Info)
ProgressLogger = MiniLogger(minlevel = LogLevel(-1))
DebugLogger = MiniLogger(minlevel = MiniLoggers.Debug)

global_logger(DebugLogger)

Base.CoreLogging.SimpleLogger(VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(21) open, 0 bytes waiting)), VSCodeServer.io_send_callback), Info, Dict{Any, Int64}())

In [29]:
mode3 = SC.Circuits.Transmon_Resonators.load("ModelSaves/Mode3/Mode3.json");

In [30]:
mode3.CandD_Ops

Dict{Any, Any} with 3 entries:
  "Dressed Mode 3 Collapse"    => Quantum Object:   type=Operator   dims=[10, 1…
  "Dressed Transmon Dephasing" => Quantum Object:   type=Operator   dims=[10, 1…
  "Dressed Transmon Collapse"  => Quantum Object:   type=Operator   dims=[10, 1…

In [28]:
mode3.Stuff["op_drive_params"]

Dict{String, Any} with 30 entries:
  "half_sb_f_1"     => Dict{String, Any}("pulse_time"=>66.2424, "Envelope Args"…
  "sb_f_1_2pi_f_3"  => Dict{String, Any}("pulse_time"=>971.249, "Envelope Args"…
  "q_e_3"           => Dict{String, Any}("pulse_time"=>71.5, "Envelope Args"=>D…
  "q_g_1_Main"      => Dict{String, Any}("pulse_time"=>100, "Envelope Args"=>Di…
  "q_g_3"           => Dict{String, Any}("pulse_time"=>100, "Envelope Args"=>Di…
  "half_q_e_0"      => Dict{String, Any}("pulse_time"=>35.48, "Envelope Args"=>…
  "q_g_3_Main"      => Dict{String, Any}("pulse_time"=>100, "Envelope Args"=>Di…
  "q_g_2"           => Dict{String, Any}("pulse_time"=>100, "Envelope Args"=>Di…
  "q_e_2"           => Dict{String, Any}("pulse_time"=>71.6, "Envelope Args"=>D…
  "half_q_e_0_Long" => Dict{String, Any}("pulse_time"=>51.495, "Envelope Args"=…
  "sb_f_2_Main"     => Dict{String, Any}("pulse_time"=>96.85, "Envelope Args"=>…
  "q_g_0"           => Dict{String, Any}("pulse_time"=>100.2, "Envelope Ar

In [37]:
ψ = mode3.dressed_states[(0,0)]

res = SC.Dynamics.RunPulseSequence(mode3, ψ, ["q_g_0"], Return = true, clean_up = true)

The Name for this run is: Operator_Sequence_2024-10-03T11:45:33.386
It is being saved at: Data/


┌ Info: Running operator q_g_0
└ @ SuperconductingCavities.Dynamics /Users/gavinrockwood/Documents/Git Repositories/SuperconductingCavities/src/Dynamics/RunOperatorSequence.jl:225
┌ Info: Running Time Evolution
└ @ SuperconductingCavities.Dynamics /Users/gavinrockwood/Documents/Git Repositories/SuperconductingCavities/src/Dynamics/RunOperatorSequence.jl:65


Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 05s (ETA: 0h 00m 00s)


┌ Info: Time Evolution Complete
└ @ SuperconductingCavities.Dynamics /Users/gavinrockwood/Documents/Git Repositories/SuperconductingCavities/src/Dynamics/RunOperatorSequence.jl:70
┌ Info: Saving Steps
└ @ SuperconductingCavities.Dynamics /Users/gavinrockwood/Documents/Git Repositories/SuperconductingCavities/src/Dynamics/RunOperatorSequence.jl:74
┌ Info: Done with Step_1
└ @ SuperconductingCavities.Dynamics /Users/gavinrockwood/Documents/Git Repositories/SuperconductingCavities/src/Dynamics/RunOperatorSequence.jl:98
┌ Info: Done With Running Sequence
└ @ SuperconductingCavities.Dynamics /Users/gavinrockwood/Documents/Git Repositories/SuperconductingCavities/src/Dynamics/RunOperatorSequence.jl:229
┌ Info: Loading Data
└ @ SuperconductingCavities.Dynamics /Users/gavinrockwood/Documents/Git Repositories/SuperconductingCavities/src/Dynamics/RunOperatorSequence.jl:231
┌ Warning: The argument dims should be a Tuple or a StaticVector for better performance. Try to use `dims = (10, 10)` or `di

Dict{Any, Any} with 2 entries:
  "times"  => Any[0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8  …  98.4, 98…
  "states" => Any[Quantum Object:   type=Ket   dims=[10, 10]   size=(100,)…

In [38]:
res

Dict{Any, Any} with 2 entries:
  "times"  => Any[0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8  …  98.4, 98…
  "states" => Any[Quantum Object:   type=Ket   dims=[10, 10]   size=(100,)…

In [46]:
for i in Iterators.product([1,2]...)
    println(i)
end

(1, 2)


In [14]:
a = [(1,2)...]

2-element Vector{Int64}:
 1
 2

In [15]:
insert!(a, 3, 4)

3-element Vector{Int64}:
 1
 2
 4